# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment
import numpy as np
import torch
from collections import deque
import random
import matplotlib.pyplot as plt

%matplotlib inline

from ddpg_agent import Agent

In [ ]:
print(random.sample([0,1,2,3,4,5,6,7,8,9], k=5))
print(np.argpartition([0,1,2,3,4,5,6,7,8,9], -2)[-2:])

In [ ]:
# check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [ ]:
# on Windows (GPU enabled)
env = UnityEnvironment(file_name="Tennis_Windows_x86_64/Tennis.exe")

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
print('Using Brain:', brain_name)

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]L
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [ ]:
for i_ep in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i_ep, np.max(scores)))

When finished, you can close the environment.

In [ ]:
env.close()

### 4. Train DDPG Agent (Multi)

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
# create agent
agent = Agent(state_size, action_size, 2)

scores_deque = deque(maxlen=100)
score_hist = []

In [ ]:
# train agent
N_EPS = 240
T_MAX = 1000 # default time limit in env is 1001
print_every  = 100

# # implement multi-agent DDPG
# def multi_ddpg(n_episodes=N_EPS, max_t=T_MAX, print_every=100):
    
#     scores_deque = deque(maxlen=print_every)
#     score_hist = []
    
for i_episode in range(1, N_EPS+1):
    env_info = env.reset(train_mode=True)[brain_name]     # turn train mode on for env
    states = env_info.vector_observations                 # get initial states
    agent.reset()                                         # reset agent
    scores = np.zeros(num_agents)                         # initialize the score (for each agent)

    for t in range(T_MAX):
        actions = [agent.act(states[0])]                # get first agents actions
        for i in range(1,num_agents):
            actions.append(agent.act(states[i]))        # get all other agents actions

        env_info = env.step(actions)[brain_name]        # send the actions to the environment
        next_states = env_info.vector_observations      # get the next state
        dones = env_info.local_done                     # see if episode has finished
        rewards = env_info.rewards                      # get reward (for each agent)
        scores += env_info.rewards                      # get total score for this episode

        # update using all agents
        for state, action, reward, next_state, done in zip(states, actions, rewards, next_states, dones):
            agent.step(state, action, reward, next_state, done)

        states = next_states

        if np.any(dones):
            print('DONE', t)
            break

    scores_deque.append(np.max(scores))
    score_hist.append(np.max(scores))
    print('\rEpisode {}\tAverage Score of Episode: {:.2f}\tAverage Score Last {}: {:.2f}'.format(i_episode, np.mean(scores), print_every, np.mean(scores_deque)), end="")
    torch.save(agent.actor_local.state_dict(), 'agent_checkpoints/checkpoint_actor.pth')
    torch.save(agent.critic_local.state_dict(), 'agent_checkpoints/checkpoint_critic.pth')
    if i_episode % print_every == 0:
        print('\rEpisode {}\tAverage Score Last {}: {:.2f}'.format(i_episode, print_every, np.mean(scores_deque)))



In [ ]:
# print rewards history
plt.rcParams['font.size'] = '16'
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

### 5. Test the Agent

Test agent in the environment.

In [ ]:
# load the saved states if testing agent

# actor_state = torch.load('agent_checkpoints/checkpoint_actor.pth')
# critic_state = torch.load('agent_checkpoints/checkpoint_critic.pth')

# agent_to_test = Agent(state_size, action_size, 4, actor_state=actor_state, critic_state=critic_state)


In [ ]:

# num_eps = 6

# for i_ep in range(1, num_eps+1):                           # play game for n_eps episodes
#     env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
#     states = env_info.vector_observations                  # get the current state (for each agent)
#     scores = np.zeros(num_agents)                          # initialize the score (for each agent)
#     while True:
# #         actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
# #         actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        
#         # test agent
#         actions = [agent.act(states[0])]                # get first agents actions
#         for i in range(1,num_agents):
#             actions.append(agent.act(states[i]))        # get all other agents actions

#         env_info = env.step(actions)[brain_name]           # send all actions to tne environment
#         next_states = env_info.vector_observations         # get next state (for each agent)
#         rewards = env_info.rewards                         # get reward (for each agent)
#         dones = env_info.local_done                        # see if episode finished
#         scores += env_info.rewards                         # update the score (for each agent)
#         states = next_states                               # roll over states to next time step
#         if np.any(dones):                                  # exit loop if episode finished
#             break
#     print('Score (max over agents) from episode {}: {}'.format(i_ep, np.max(scores)))